<a href="https://colab.research.google.com/github/vinicius-marques1/Cognizant_Cloud_Data_Engineer/blob/main/fundamentos_ETL_com_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pandera

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd
import pandera as pa

In [12]:
valores_ausentes = ['**','****','*****','###!','####','NULL']
df = pd.read_csv('/content/drive/MyDrive/bases_de_dados/ocorrencia.csv', sep=';', parse_dates=['ocorrencia_dia'], dayfirst=True, na_values=valores_ausentes)
df.head(10)

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,40211,40211,INCIDENTE,RIO DE JANEIRO,RJ,NaN,2010-01-03,12:00:00,0
1,40349,40349,INCIDENTE,BELÉM,PA,SBBE,2010-01-03,11:05:00,0
2,40351,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-01-03,03:00:00,0
3,39527,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,NaN,2010-01-04,17:30:00,0
4,40324,40324,INCIDENTE,PELOTAS,RS,SBPK,2010-01-05,19:25:00,0
5,39807,39807,INCIDENTE,SALVADOR,BA,NaN,2010-01-06,17:53:00,0
6,40215,40215,INCIDENTE,COARI,AM,SBUY,2010-01-07,18:40:00,0
7,39707,39707,INCIDENTE GRAVE,CANUTAMA,AM,NaN,2010-01-09,12:30:00,3
8,39156,39156,INCIDENTE GRAVE,CASCAVEL,PR,SBCA,2010-01-10,23:15:00,2
9,39711,39711,INCIDENTE GRAVE,PARÁ DE MINAS,MG,NaN,2010-01-10,20:00:00,0


In [16]:
schema = pa.DataFrameSchema(
    columns={
        'codigo_ocorrencia': pa.Column(pa.Int),
        'codigo_ocorrencia2': pa.Column(pa.Int),
        'ocorrencia_classificacao': pa.Column(pa.String),
        'ocorrencia_cidade': pa.Column(pa.String),
        'ocorrencia_uf': pa.Column(pa.String, pa.Check.str_length(2,2), nullable=True),
        'ocorrencia_aerodromo': pa.Column(pa.String, nullable=True),
        'ocorrencia_dia': pa.Column(pa.DateTime),
        'ocorrencia_hora': pa.Column(pa.String, nullable=True),
        'total_recomendacoes': pa.Column(pa.Int)
    }
)

In [17]:
schema.validate(df)

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,40211,40211,INCIDENTE,RIO DE JANEIRO,RJ,NaN,2010-01-03,12:00:00,0
1,40349,40349,INCIDENTE,BELÉM,PA,SBBE,2010-01-03,11:05:00,0
2,40351,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-01-03,03:00:00,0
3,39527,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,NaN,2010-01-04,17:30:00,0
4,40324,40324,INCIDENTE,PELOTAS,RS,SBPK,2010-01-05,19:25:00,0
...,...,...,...,...,...,...,...,...,...
6109,80245,80245,INCIDENTE,RECIFE,PE,SBRF,2021-08-15,15:05:00,0
6110,80247,80247,INCIDENTE,PORTO VELHO,RO,SBPV,2021-08-16,02:30:00,0
6111,80251,80251,INCIDENTE,BELÉM,PA,SBBE,2021-08-17,15:45:00,0
6112,80257,80257,ACIDENTE,DOM AQUINO,MT,NaN,2021-08-18,16:00:00,0


In [8]:
df['codigo_ocorrencia'].is_unique

True

In [18]:
df.dtypes

codigo_ocorrencia                    int64
codigo_ocorrencia2                   int64
ocorrencia_classificacao            object
ocorrencia_cidade                   object
ocorrencia_uf                       object
ocorrencia_aerodromo                object
ocorrencia_dia              datetime64[ns]
ocorrencia_hora                     object
total_recomendacoes                  int64
dtype: object

In [23]:
df.describe()

,codigo_ocorrencia,codigo_ocorrencia2,total_recomendacoes
count,6114.000000,6114.000000,6114.000000
mean,59775.058063,59775.058063,0.307164
std,14552.373693,14552.373693,1.143202
min,39115.000000,39115.000000,0.000000
25%,46586.250000,46586.250000,0.000000
50%,53110.500000,53110.500000,0.000000
75%,78142.500000,78142.500000,0.000000
max,80259.000000,80259.000000,23.000000


In [24]:
df.iloc[-1]

codigo_ocorrencia                         80259
codigo_ocorrencia2                        80259
ocorrencia_classificacao              INCIDENTE
ocorrencia_cidade                      CAMPINAS
ocorrencia_uf                                SP
ocorrencia_aerodromo                       SBKP
ocorrencia_dia              2021-08-18 00:00:00
ocorrencia_hora                        12:45:00
total_recomendacoes                           0
Name: 6113, dtype: object

In [27]:
df.isna().sum()

codigo_ocorrencia              0
codigo_ocorrencia2             0
ocorrencia_classificacao       0
ocorrencia_cidade              0
ocorrencia_uf                  2
ocorrencia_aerodromo        2456
ocorrencia_dia                 0
ocorrencia_hora                1
total_recomendacoes            0
dtype: int64

In [30]:
filtro = df.ocorrencia_uf.isnull()
df.loc[filtro]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
2227,49474,49474,ACIDENTE,ÁGUAS INTERNACIONAIS,NaN,NaN,2013-09-02,02:54:00,0
5855,79956,79956,ACIDENTE,NÃO IDENTIFICADA,NaN,NaN,2021-02-26,11:00:00,0


In [31]:
df.count()

codigo_ocorrencia           6114
codigo_ocorrencia2          6114
ocorrencia_classificacao    6114
ocorrencia_cidade           6114
ocorrencia_uf               6112
ocorrencia_aerodromo        3658
ocorrencia_dia              6114
ocorrencia_hora             6113
total_recomendacoes         6114
dtype: int64

In [34]:
# Ocorrências com mais de 10 recomendações
filtro = df.total_recomendacoes > 10
df.loc[filtro, ['ocorrencia_cidade', 'total_recomendacoes']]

,ocorrencia_cidade,total_recomendacoes
63,SÃO PAULO,11
326,RIO DE JANEIRO,19
530,SÃO PAULO,11
727,ITÁPOLIS,11
872,RECIFE,23
884,SÃO JOSÉ DOS PINHAIS,11
1062,UBERLÂNDIA,20
1972,BRASÍLIA,11
2797,SANTOS,13
3932,VITÓRIA,12


In [36]:
# Ocorrências cuja classificação == INCIDENTE GRAVE
filtro = df.ocorrencia_classificacao == 'INCIDENTE GRAVE'
df.loc[filtro]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
7,39707,39707,INCIDENTE GRAVE,CANUTAMA,AM,NaN,2010-01-09,12:30:00,3
8,39156,39156,INCIDENTE GRAVE,CASCAVEL,PR,SBCA,2010-01-10,23:15:00,2
9,39711,39711,INCIDENTE GRAVE,PARÁ DE MINAS,MG,NaN,2010-01-10,20:00:00,0
29,39709,39709,INCIDENTE GRAVE,CURITIBA,PR,SBBI,2010-01-23,16:36:00,2
35,39487,39487,INCIDENTE GRAVE,PALMAS,TO,SBPJ,2010-01-26,16:50:00,0
...,...,...,...,...,...,...,...,...,...
6090,80221,80221,INCIDENTE GRAVE,MARAÚ,BA,SIRI,2021-07-31,13:15:00,0
6095,80228,80228,INCIDENTE GRAVE,ELDORADO DO SUL,RS,SIXE,2021-08-05,15:55:00,0
6097,80231,80231,INCIDENTE GRAVE,CAXIAS DO SUL,RS,SBCX,2021-08-06,16:59:00,0
6098,80248,80248,INCIDENTE GRAVE,CHAPADÃO DO SUL,MS,SSCD,2021-08-06,20:00:00,0


In [39]:
# Ocorrências cuja classificação == INCIDENTE GRAVE e estado == SP
filtro1 = df.ocorrencia_classificacao == 'INCIDENTE GRAVE'
filtro2 = df.ocorrencia_uf == 'SP'
df.loc[filtro1 & filtro2]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
136,40270,40270,INCIDENTE GRAVE,PIRASSUNUNGA,SP,SDPY,2010-04-15,20:15:00,0
171,40620,40620,INCIDENTE GRAVE,GUARULHOS,SP,SBGR,2010-05-06,15:50:00,0
219,41411,41411,INCIDENTE GRAVE,SÃO PAULO,SP,SBMT,2010-06-08,19:30:00,2
297,42289,42289,INCIDENTE GRAVE,SÃO PAULO,SP,SBMT,2010-07-29,20:06:00,0
314,42323,42323,INCIDENTE GRAVE,SÃO JOSÉ DO RIO PRETO,SP,SBSR,2010-08-05,12:01:00,0
...,...,...,...,...,...,...,...,...,...
5949,80056,80056,INCIDENTE GRAVE,FERNANDÓPOLIS,SP,NaN,2021-04-28,11:20:00,0
6000,80114,80114,INCIDENTE GRAVE,LINS,SP,SWXQ,2021-06-05,19:43:00,0
6007,80124,80124,INCIDENTE GRAVE,SÃO PAULO,SP,SBSP,2021-06-10,20:30:00,0
6081,80200,80200,INCIDENTE GRAVE,BRAGANÇA PAULISTA,SP,SDVH,2021-07-25,12:25:00,0


In [40]:
# Ocorrências cuja classificação == INCIDENTE GRAVE ou estado == SP
filtro1 = df.ocorrencia_classificacao == 'INCIDENTE GRAVE'
filtro2 = df.ocorrencia_uf == 'SP'
df.loc[filtro1 | filtro2]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
7,39707,39707,INCIDENTE GRAVE,CANUTAMA,AM,NaN,2010-01-09,12:30:00,3
8,39156,39156,INCIDENTE GRAVE,CASCAVEL,PR,SBCA,2010-01-10,23:15:00,2
9,39711,39711,INCIDENTE GRAVE,PARÁ DE MINAS,MG,NaN,2010-01-10,20:00:00,0
11,40069,40069,ACIDENTE,SÃO PAULO,SP,SBMT,2010-01-10,14:50:00,8
16,39809,39809,INCIDENTE,SÃO PAULO,SP,NaN,2010-01-15,15:00:00,0
...,...,...,...,...,...,...,...,...,...
6097,80231,80231,INCIDENTE GRAVE,CAXIAS DO SUL,RS,SBCX,2021-08-06,16:59:00,0
6098,80248,80248,INCIDENTE GRAVE,CHAPADÃO DO SUL,MS,SSCD,2021-08-06,20:00:00,0
6104,80238,80238,INCIDENTE GRAVE,VOTUPORANGA,SP,SDVG,2021-08-11,19:09:00,0
6108,80244,80244,INCIDENTE,BRAGANÇA PAULISTA,SP,SBBP,2021-08-15,15:20:00,0


In [41]:
# Ocorrências cuja (classificação == INCIDENTE GRAVE ou classificação == INCIDENTE) e o estado == SP
filtro1 = (df.ocorrencia_classificacao == 'INCIDENTE GRAVE') | (df.ocorrencia_classificacao == 'INCIDENTE')
filtro2 = df.ocorrencia_uf == 'SP'
df.loc[filtro1 & filtro2]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
16,39809,39809,INCIDENTE,SÃO PAULO,SP,NaN,2010-01-15,15:00:00,0
18,39828,39828,INCIDENTE,SANTOS,SP,NaN,2010-01-15,17:45:00,0
26,39847,39847,INCIDENTE,SOROCABA,SP,NaN,2010-01-20,13:10:00,0
27,39768,39768,INCIDENTE,CAMPINAS,SP,NaN,2010-01-21,20:45:00,0
33,39848,39848,INCIDENTE,GUARULHOS,SP,SBGR,2010-01-25,17:05:00,0
...,...,...,...,...,...,...,...,...,...
6084,80234,80234,INCIDENTE,SÃO PAULO,SP,SBSP,2021-07-26,13:20:00,0
6085,80208,80208,INCIDENTE,SÃO PAULO,SP,SBMT,2021-07-27,19:45:00,0
6104,80238,80238,INCIDENTE GRAVE,VOTUPORANGA,SP,SDVG,2021-08-11,19:09:00,0
6108,80244,80244,INCIDENTE,BRAGANÇA PAULISTA,SP,SBBP,2021-08-15,15:20:00,0


In [43]:
# Ocorrências cuja (classificação == INCIDENTE GRAVE ou classificação == INCIDENTE) e o estado == SP
filtro1 = (df.ocorrencia_classificacao.isin(['INCIDENTE GRAVE', 'INCIDENTE']))
filtro2 = df.ocorrencia_uf == 'SP'
df.loc[filtro1 & filtro2]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
16,39809,39809,INCIDENTE,SÃO PAULO,SP,NaN,2010-01-15,15:00:00,0
18,39828,39828,INCIDENTE,SANTOS,SP,NaN,2010-01-15,17:45:00,0
26,39847,39847,INCIDENTE,SOROCABA,SP,NaN,2010-01-20,13:10:00,0
27,39768,39768,INCIDENTE,CAMPINAS,SP,NaN,2010-01-21,20:45:00,0
33,39848,39848,INCIDENTE,GUARULHOS,SP,SBGR,2010-01-25,17:05:00,0
...,...,...,...,...,...,...,...,...,...
6084,80234,80234,INCIDENTE,SÃO PAULO,SP,SBSP,2021-07-26,13:20:00,0
6085,80208,80208,INCIDENTE,SÃO PAULO,SP,SBMT,2021-07-27,19:45:00,0
6104,80238,80238,INCIDENTE GRAVE,VOTUPORANGA,SP,SDVG,2021-08-11,19:09:00,0
6108,80244,80244,INCIDENTE,BRAGANÇA PAULISTA,SP,SBBP,2021-08-15,15:20:00,0


In [46]:
# Ocorrências cuja cidade começa com a letra C
filtro = df.ocorrencia_cidade.str[0] == 'C'
df.loc[filtro]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
6,40215,40215,INCIDENTE,COARI,AM,SBUY,2010-01-07,18:40:00,0
7,39707,39707,INCIDENTE GRAVE,CANUTAMA,AM,NaN,2010-01-09,12:30:00,3
8,39156,39156,INCIDENTE GRAVE,CASCAVEL,PR,SBCA,2010-01-10,23:15:00,2
15,39315,39315,ACIDENTE,CANUTAMA,AM,NaN,2010-01-15,21:22:00,0
20,40310,40310,INCIDENTE,CRUZEIRO DO SUL,AC,SBCZ,2010-01-18,19:32:00,0
...,...,...,...,...,...,...,...,...,...
6094,80225,80225,INCIDENTE,CURITIBA,PR,SBBI,2021-08-04,13:45:00,0
6097,80231,80231,INCIDENTE GRAVE,CAXIAS DO SUL,RS,SBCX,2021-08-06,16:59:00,0
6098,80248,80248,INCIDENTE GRAVE,CHAPADÃO DO SUL,MS,SSCD,2021-08-06,20:00:00,0
6100,80235,80235,INCIDENTE,CORUMBÁ,MS,SBCR,2021-08-09,20:55:00,0


In [47]:
# Ocorrências cuja cidade termina com MA
filtro = df.ocorrencia_cidade.str[-2:] == 'MA'
df.loc[filtro]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
7,39707,39707,INCIDENTE GRAVE,CANUTAMA,AM,NaN,2010-01-09,12:30:00,3
15,39315,39315,ACIDENTE,CANUTAMA,AM,NaN,2010-01-15,21:22:00,0
408,43062,43062,ACIDENTE,NOVO GAMA,GO,NaN,2010-09-25,16:30:00,1
688,44198,44198,ACIDENTE,BURITAMA,SP,NaN,2011-03-19,11:18:00,0
1035,51785,51785,INCIDENTE,BURITAMA,SP,SIBX,2011-10-25,19:15:00,0
1377,45939,45939,ACIDENTE,UMUARAMA,PR,NaN,2012-05-30,19:00:00,0
1401,46180,46180,ACIDENTE,AURIFLAMA,SP,NaN,2012-06-16,19:30:00,0
2099,49113,49113,INCIDENTE GRAVE,NOVA LIMA,MG,NaN,2013-06-15,12:00:00,4
2102,48799,48799,ACIDENTE,CANUTAMA,AM,NaN,2013-06-18,21:30:00,0
2646,80059,80059,INCIDENTE,IVINHEMA,MS,NaN,2014-04-27,11:30:00,0


In [48]:
# Ocorrências cuja cidade contem MA
filtro = df.ocorrencia_cidade.str.contains('MA')
df.loc[filtro]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
7,39707,39707,INCIDENTE GRAVE,CANUTAMA,AM,NaN,2010-01-09,12:30:00,3
13,39507,39507,ACIDENTE,PRIMAVERA DO LESTE,MT,NaN,2010-01-11,17:00:00,2
15,39315,39315,ACIDENTE,CANUTAMA,AM,NaN,2010-01-15,21:22:00,0
35,39487,39487,INCIDENTE GRAVE,PALMAS,TO,SBPJ,2010-01-26,16:50:00,0
78,39916,39916,INCIDENTE GRAVE,MANAUS,AM,SBEG,2010-03-07,19:35:00,2
...,...,...,...,...,...,...,...,...,...
6075,80205,80205,INCIDENTE,MANAUS,AM,SBEG,2021-07-23,11:15:00,0
6086,80212,80212,INCIDENTE,AMAPÁ,AP,NaN,2021-07-28,18:00:00,0
6090,80221,80221,INCIDENTE GRAVE,MARAÚ,BA,SIRI,2021-07-31,13:15:00,0
6093,80227,80227,ACIDENTE,UMUARAMA,PR,NaN,2021-08-02,14:30:00,0


In [50]:
# Ocorrências cuja cidade contem MA ou AL
filtro = df.ocorrencia_cidade.str.contains('MA|AL')
df.loc[filtro]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
5,39807,39807,INCIDENTE,SALVADOR,BA,NaN,2010-01-06,17:53:00,0
7,39707,39707,INCIDENTE GRAVE,CANUTAMA,AM,NaN,2010-01-09,12:30:00,3
13,39507,39507,ACIDENTE,PRIMAVERA DO LESTE,MT,NaN,2010-01-11,17:00:00,2
15,39315,39315,ACIDENTE,CANUTAMA,AM,NaN,2010-01-15,21:22:00,0
17,39810,39810,INCIDENTE,SALVADOR,BA,NaN,2010-01-15,18:23:00,0
...,...,...,...,...,...,...,...,...,...
6075,80205,80205,INCIDENTE,MANAUS,AM,SBEG,2021-07-23,11:15:00,0
6086,80212,80212,INCIDENTE,AMAPÁ,AP,NaN,2021-07-28,18:00:00,0
6090,80221,80221,INCIDENTE GRAVE,MARAÚ,BA,SIRI,2021-07-31,13:15:00,0
6093,80227,80227,ACIDENTE,UMUARAMA,PR,NaN,2021-08-02,14:30:00,0


In [51]:
# Ocorrências do ano de 2015
filtro = df.ocorrencia_dia.dt.year == 2015
df.loc[filtro]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
2998,52992,52992,INCIDENTE GRAVE,SALVADOR,BA,SBSV,2015-01-01,11:40:00,3
2999,52979,52979,ACIDENTE,IVINHEMA,MS,NaN,2015-01-02,14:50:00,0
3000,53073,53073,INCIDENTE,TEFÉ,AM,SBTF,2015-01-02,16:28:00,0
3001,53074,53074,INCIDENTE,BELO HORIZONTE,MG,SBBH,2015-01-02,20:34:00,0
3002,52976,52976,ACIDENTE,TOLEDO,PR,SBTD,2015-01-04,22:04:00,6
...,...,...,...,...,...,...,...,...,...
3464,60632,60632,INCIDENTE GRAVE,ITABERÁ,SP,NaN,2015-12-24,14:00:00,0
3465,60600,60600,INCIDENTE,GUARULHOS,SP,SBGR,2015-12-25,19:00:00,0
3466,60642,60642,INCIDENTE,SÃO FRANCISCO DO SUL,SC,SSSS,2015-12-26,16:00:00,0
3467,60631,60631,ACIDENTE,MAÇAMBARÁ,RS,NaN,2015-12-28,19:00:00,2


In [53]:
# Ocorrências de dezembro de 2015
filtro = (df.ocorrencia_dia.dt.year == 2015) & (df.ocorrencia_dia.dt.month == 12)
df.loc[filtro]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
3430,53573,53573,INCIDENTE,GUARULHOS,SP,SBGR,2015-12-01,02:48:00,0
3431,60601,60601,INCIDENTE,PALMAS,TO,SBPJ,2015-12-01,16:05:00,0
3432,53634,53634,INCIDENTE,PALMAS,TO,SBPJ,2015-12-02,17:45:00,0
3433,53636,53636,INCIDENTE,JUNDIAÍ,SP,SBJD,2015-12-02,17:42:00,0
3434,53575,53575,INCIDENTE,CAMPOS DOS GOYTACAZES,RJ,SBFS,2015-12-03,10:50:00,0
3435,60637,60637,INCIDENTE,BELO HORIZONTE,MG,SBBH,2015-12-03,16:47:00,0
3436,53625,53625,ACIDENTE,TRINDADE,GO,NaN,2015-12-06,13:10:00,3
3437,53626,53626,ACIDENTE,AMERICANA,SP,SDAI,2015-12-06,15:00:00,1
3438,53628,53628,ACIDENTE,AGUAÍ,SP,NaN,2015-12-08,14:30:00,1
3439,53629,53629,INCIDENTE GRAVE,JALES,SP,SDJL,2015-12-08,10:20:00,0


In [56]:
# Ocorrências do ano de dezembro de 2015 entre di 2 e 9
filtro_ano = df.ocorrencia_dia.dt.year == 2015 
filtro_mes = df.ocorrencia_dia.dt.month == 12
filtro_dia = (df.ocorrencia_dia.dt.day > 2) & (df.ocorrencia_dia.dt.day < 9)
df.loc[filtro_ano & filtro_mes & filtro_dia]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
3434,53575,53575,INCIDENTE,CAMPOS DOS GOYTACAZES,RJ,SBFS,2015-12-03,10:50:00,0
3435,60637,60637,INCIDENTE,BELO HORIZONTE,MG,SBBH,2015-12-03,16:47:00,0
3436,53625,53625,ACIDENTE,TRINDADE,GO,NaN,2015-12-06,13:10:00,3
3437,53626,53626,ACIDENTE,AMERICANA,SP,SDAI,2015-12-06,15:00:00,1
3438,53628,53628,ACIDENTE,AGUAÍ,SP,NaN,2015-12-08,14:30:00,1
3439,53629,53629,INCIDENTE GRAVE,JALES,SP,SDJL,2015-12-08,10:20:00,0
3440,53631,53631,INCIDENTE,CAMPINAS,SP,SBKP,2015-12-08,16:19:00,0
3441,60636,60636,INCIDENTE,CAXIAS DO SUL,RS,SBCX,2015-12-08,13:00:00,0


In [58]:
# Crição de uma nova coluna
df['ocorrencia_dia_hora'] = pd.to_datetime(df.ocorrencia_dia.astype(str) + ' ' + df.ocorrencia_hora)

In [60]:
df.dtypes

codigo_ocorrencia                    int64
codigo_ocorrencia2                   int64
ocorrencia_classificacao            object
ocorrencia_cidade                   object
ocorrencia_uf                       object
ocorrencia_aerodromo                object
ocorrencia_dia              datetime64[ns]
ocorrencia_hora                     object
total_recomendacoes                  int64
ocorrencia_dia_hora         datetime64[ns]
dtype: object

In [64]:
filtro = (df.ocorrencia_dia_hora >= '2010-01-03 11:05:00') & (df.ocorrencia_dia_hora <= '2010-01-08 13:00:00')
df.loc[filtro]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes,ocorrencia_dia_hora
0,40211,40211,INCIDENTE,RIO DE JANEIRO,RJ,NaN,2010-01-03,12:00:00,0,2010-01-03 12:00:00
1,40349,40349,INCIDENTE,BELÉM,PA,SBBE,2010-01-03,11:05:00,0,2010-01-03 11:05:00
3,39527,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,NaN,2010-01-04,17:30:00,0,2010-01-04 17:30:00
4,40324,40324,INCIDENTE,PELOTAS,RS,SBPK,2010-01-05,19:25:00,0,2010-01-05 19:25:00
5,39807,39807,INCIDENTE,SALVADOR,BA,NaN,2010-01-06,17:53:00,0,2010-01-06 17:53:00
6,40215,40215,INCIDENTE,COARI,AM,SBUY,2010-01-07,18:40:00,0,2010-01-07 18:40:00


In [65]:
# Ocorrências do ano 2015 e mês 3
filtro = (df.ocorrencia_dia.dt.year == 2015) & (df.ocorrencia_dia.dt.month == 3)
df_201503 = df.loc[filtro]

In [66]:
df_201503.count()

codigo_ocorrencia           37
codigo_ocorrencia2          37
ocorrencia_classificacao    37
ocorrencia_cidade           37
ocorrencia_uf               37
ocorrencia_aerodromo        21
ocorrencia_dia              37
ocorrencia_hora             37
total_recomendacoes         37
ocorrencia_dia_hora         37
dtype: int64

In [70]:
# Agrupando por ocorrencia_classificacao e contando por codigo_ocorrencia
df_201503.groupby('ocorrencia_classificacao').codigo_ocorrencia.count()

ocorrencia_classificacao
ACIDENTE           15
INCIDENTE          17
INCIDENTE GRAVE     5
Name: codigo_ocorrencia, dtype: int64

In [71]:
# contando por ocorrencia_aerodromo (Não recomendado porque nessa coluna exitem valores Na)
df_201503.groupby('ocorrencia_classificacao').ocorrencia_aerodromo.count()

ocorrencia_classificacao
ACIDENTE            5
INCIDENTE          14
INCIDENTE GRAVE     2
Name: ocorrencia_aerodromo, dtype: int64

In [77]:
df_201503.groupby(['ocorrencia_classificacao']).size().sort_values(ascending=False)

ocorrencia_classificacao
INCIDENTE          17
ACIDENTE           15
INCIDENTE GRAVE     5
dtype: int64

In [79]:
# geração de um novo DataFrame para os estados do sudeste e no ano de 2010
filtro = (df.ocorrencia_dia.dt.year == 2010) & (df.ocorrencia_uf.isin(['SP', 'MG', 'ES', 'RJ']))
df_sudeste2010 = df.loc[filtro]
df_sudeste2010

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes,ocorrencia_dia_hora
0,40211,40211,INCIDENTE,RIO DE JANEIRO,RJ,NaN,2010-01-03,12:00:00,0,2010-01-03 12:00:00
2,40351,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-01-03,03:00:00,0,2010-01-03 03:00:00
9,39711,39711,INCIDENTE GRAVE,PARÁ DE MINAS,MG,NaN,2010-01-10,20:00:00,0,2010-01-10 20:00:00
11,40069,40069,ACIDENTE,SÃO PAULO,SP,SBMT,2010-01-10,14:50:00,8,2010-01-10 14:50:00
16,39809,39809,INCIDENTE,SÃO PAULO,SP,NaN,2010-01-15,15:00:00,0,2010-01-15 15:00:00
...,...,...,...,...,...,...,...,...,...,...
541,43722,43722,ACIDENTE,CAJOBI,SP,NaN,2010-12-23,11:00:00,0,2010-12-23 11:00:00
544,43548,43548,INCIDENTE,SÃO PAULO,SP,SBSP,2010-12-26,22:10:00,0,2010-12-26 22:10:00
548,43724,43724,ACIDENTE,COSMÓPOLIS,SP,NaN,2010-12-29,12:00:00,0,2010-12-29 12:00:00
550,43730,43730,INCIDENTE,VITÓRIA,ES,NaN,2010-12-30,18:10:00,0,2010-12-30 18:10:00


In [80]:
df_sudeste2010.groupby('ocorrencia_classificacao').size()

ocorrencia_classificacao
ACIDENTE            40
INCIDENTE          170
INCIDENTE GRAVE     18
dtype: int64

In [81]:
df_sudeste2010.groupby(['ocorrencia_classificacao', 'ocorrencia_uf']).size()

ocorrencia_classificacao  ocorrencia_uf
ACIDENTE                  MG                7
                          RJ                9
                          SP               24
INCIDENTE                 ES                5
                          MG               39
                          RJ               52
                          SP               74
INCIDENTE GRAVE           MG                5
                          RJ                4
                          SP                9
dtype: int64

In [82]:
df_sudeste2010.groupby(['ocorrencia_uf', 'ocorrencia_classificacao']).size()

ocorrencia_uf  ocorrencia_classificacao
ES             INCIDENTE                    5
MG             ACIDENTE                     7
               INCIDENTE                   39
               INCIDENTE GRAVE              5
RJ             ACIDENTE                     9
               INCIDENTE                   52
               INCIDENTE GRAVE              4
SP             ACIDENTE                    24
               INCIDENTE                   74
               INCIDENTE GRAVE              9
dtype: int64

In [84]:
df_sudeste2010.groupby(['ocorrencia_cidade']).size().sort_values(ascending=False)

ocorrencia_cidade
RIO DE JANEIRO           47
SÃO PAULO                33
GUARULHOS                18
BELO HORIZONTE           17
CAMPINAS                 13
MACAÉ                    12
CONFINS                   7
UBERLÂNDIA                5
VITÓRIA                   4
JUIZ DE FORA              4
SÃO JOSÉ DO RIO PRETO     3
SÃO JOSÉ DOS CAMPOS       3
MARÍLIA                   3
ARARAQUARA                2
MONTES CLAROS             2
CAMPOS DOS GOYTACAZES     2
DIVINÓPOLIS               2
IPATINGA                  2
ITANHAÉM                  2
JUNDIAÍ                   2
POÇOS DE CALDAS           2
RIBEIRÃO PRETO            2
SOROCABA                  2
UBERABA                   2
BAURU                     2
AREALVA                   2
BOITUVA                   1
BOCAINA                   1
BOM JESUS DO AMPARO       1
BOM JESUS DO GALHO        1
BEBEDOURO                 1
BRAGANÇA PAULISTA         1
BROTAS                    1
CAJOBI                    1
ARRAIAL DO CABO           1
CA

In [87]:
# total de recomendações no Rio de Janeiro
filtro = df_sudeste2010.ocorrencia_cidade == 'RIO DE JANEIRO'
df_sudeste2010.loc[filtro].total_recomendacoes.sum()

25

In [93]:
# Agrupando por cidade e somando total_recomendações (Só para cidades com total de recomendações > 0)
df_sudeste2010.loc[df_sudeste2010.total_recomendacoes > 0].groupby(['ocorrencia_cidade']).total_recomendacoes.sum().sort_values(ascending=False)

ocorrencia_cidade
SÃO PAULO                41
RIO DE JANEIRO           25
CAMANDUCAIA               9
BRAGANÇA PAULISTA         9
RESENDE                   8
JUNDIAÍ                   7
TIMÓTEO                   3
NOVO HORIZONTE            3
IPERÓ                     3
SÃO BERNARDO DO CAMPO     2
BELO HORIZONTE            2
ITANHAÉM                  1
BOM JESUS DO GALHO        1
BOITUVA                   1
Name: total_recomendacoes, dtype: int64

In [95]:
df_sudeste2010.groupby(['ocorrencia_aerodromo'], dropna=False).total_recomendacoes.sum()

ocorrencia_aerodromo
SBAE     0
SBAQ     0
SBBH     0
SBBP     9
SBBU     0
SBCF     0
SBGL     3
SBGR     0
SBIP     0
SBJD     7
SBJR     3
SBKP     0
SBME     0
SBMK     0
SBML     0
SBMT    27
SBPC     0
SBPR     1
SBRJ    19
SBRP     0
SBSJ     0
SBSP     0
SBSR     0
SBUL     0
SBUR     0
SBVT     0
SDAM     0
SDOI     1
SDPW     0
SDPY     0
SNDT     0
SNDV     0
SNXT     0
NaN     45
Name: total_recomendacoes, dtype: int64

In [98]:
filtro_total_re = df_sudeste2010.total_recomendacoes > 0
df_sudeste2010.loc[filtro_total_re].groupby(['ocorrencia_cidade', df_sudeste2010.ocorrencia_dia.dt.month]).total_recomendacoes.sum()

ocorrencia_cidade      ocorrencia_dia
BELO HORIZONTE         2                  1
                       3                  1
BOITUVA                7                  1
BOM JESUS DO GALHO     12                 1
BRAGANÇA PAULISTA      11                 9
CAMANDUCAIA            7                  9
IPERÓ                  1                  3
ITANHAÉM               9                  1
JUNDIAÍ                5                  5
                       6                  2
NOVO HORIZONTE         7                  3
RESENDE                4                  8
RIO DE JANEIRO         4                  3
                       7                  3
                       8                 19
SÃO BERNARDO DO CAMPO  1                  2
SÃO PAULO              1                  8
                       2                 14
                       6                  2
                       11                 6
                       12                11
TIMÓTEO                2              